In [1]:
# data analysis and wrangling
import pandas as pd
import numpy as np
import random as rnd
import xgboost as xg
import operator
# visualization
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

# machine learning
from sklearn.externals import joblib
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC, LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import Perceptron
from sklearn.linear_model import SGDClassifier
from sklearn.tree import DecisionTreeClassifier

c:\users\123\appdata\local\programs\python\python35\lib\site-packages\sklearn\cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [15]:
a=pd.read_csv(r'C:\Users\123\Desktop\caixun\Caixun (1)\lab_data_full_average_1_0.csv')


In [16]:
a.describe()

,Unnamed: 0,user_id,is_actpay,ispay_1M,ispay_2M,ispay_3M,ispay_4M,cur_arrears_1M,cur_arrears_2M,cur_arrears_3M,...,login_times_3M,read_times_1M,read_times_2M,read_times_3M,click_times_1M,click_times_2M,click_times_3M,mpost_actcnt_1M,mpost_actcnt_2M,mpost_actcnt_3M
count,6000.000000,6000.000000,6000.000000,6000.000000,6000.000000,6000.000000,6000.000000,6000.000000,6000.000000,6000.000000,...,6000.000000,6000.000000,6000.000000,6000.000000,6000.000000,6000.000000,6000.000000,6000.000000,6000.000000,6000.000000
mean,2999.500000,3000.500000,0.500000,0.101000,0.081167,0.079000,0.252167,27.646307,28.381636,11.885466,...,13.590895,29.954598,27.585925,28.208898,33.741481,31.971811,36.474510,0.005333,0.004167,0.003500
std,1732.195139,1732.195139,0.500042,0.301354,0.273114,0.269761,0.434293,153.197891,154.977097,77.219363,...,42.453728,180.012294,155.719912,117.172820,112.583301,122.834218,193.828473,0.079410,0.064420,0.066996
min,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000
25%,1499.750000,1500.750000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,3.000000,5.000000,5.000000,4.000000,10.000000,8.000000,9.000000,0.000000,0.000000,0.000000
50%,2999.500000,3000.500000,0.500000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,13.590895,29.954598,27.585925,28.208898,33.741481,31.971811,36.474510,0.000000,0.000000,0.000000
75%,4499.250000,4500.250000,1.000000,0.000000,0.000000,0.000000,1.000000,27.646307,28.381636,0.000000,...,13.590895,29.954598,27.585925,28.208898,33.741481,31.971811,36.474510,0.000000,0.000000,0.000000
max,5999.000000,6000.000000,1.000000,1.000000,1.000000,1.000000,1.000000,4903.230000,4612.000000,2784.820000,...,2219.000000,10519.000000,9723.000000,4664.000000,6783.000000,7613.000000,13482.000000,2.000000,1.000000,2.000000


In [17]:
y = a.is_actpay

del a['is_actpay']
del a['user_id']
del a['ispay_1M']
del a['ispay_2M']
del a['ispay_3M']
del a['ispay_4M']
del a['Unnamed: 0']


In [12]:
a

,cur_arrears_1M,cur_arrears_2M,cur_arrears_3M,arpu_1M,arpu_2M,arpu_3M,reduce_fee_1M,reduce_fee_2M,reduce_fee_3M,prdct_fee_1M,...,login_times_3M,read_times_1M,read_times_2M,read_times_3M,click_times_1M,click_times_2M,click_times_3M,mpost_actcnt_1M,mpost_actcnt_2M,mpost_actcnt_3M
0,28.610000,28.610000,12.740000,123.22,116.92,121.80,26.930000,27.830000,29.410000,86.350000,...,18.118463,46.346102,42.169137,41.623171,48.703046,46.786341,52.324146,0,0,0
1,0.000000,0.230000,0.000000,28.15,15.23,72.24,37.890000,12.450000,0.000000,0.000000,...,11.000000,7.557392,7.694853,8.581336,8.000000,10.000000,12.000000,0,0,0
2,51.760000,0.000000,0.000000,24.39,26.38,31.71,30.200000,29.600000,30.200000,12.000000,...,7.118993,7.557392,1.000000,8.581336,12.763345,1.000000,13.399679,0,0,0
3,27.362794,27.324078,0.000000,108.30,102.20,20.36,27.474975,27.580515,0.000000,70.075153,...,1.000000,7.557392,7.694853,2.000000,12.763345,11.780529,2.000000,0,0,0
4,27.362794,27.324078,0.000000,108.30,102.20,28.33,27.474975,27.580515,20.000000,70.075153,...,7.118993,7.557392,7.694853,8.581336,12.763345,11.780529,13.399679,0,0,0
5,28.610000,28.610000,12.740000,123.22,116.92,121.80,26.930000,27.830000,29.410000,86.350000,...,42.000000,46.346102,165.000000,43.000000,48.703046,124.000000,48.000000,0,0,0
6,28.610000,28.610000,12.740000,123.22,116.92,121.80,26.930000,27.830000,29.410000,86.350000,...,18.118463,11.000000,42.169137,41.623171,19.000000,46.786341,52.324146,0,0,0
7,28.610000,28.610000,12.740000,123.22,116.92,121.80,26.930000,27.830000,29.410000,86.350000,...,20.000000,16.000000,42.169137,20.000000,11.000000,46.786341,20.000000,0,0,0
8,27.362794,0.000000,0.000000,108.30,30.81,37.72,27.474975,0.000000,0.000000,70.075153,...,7.118993,7.557392,7.694853,8.581336,12.763345,11.780529,13.399679,0,0,0
9,28.610000,28.610000,12.740000,123.22,116.92,121.80,26.930000,27.830000,29.410000,86.350000,...,18.118463,46.346102,42.169137,41.623171,48.703046,46.786341,52.324146,0,0,0


In [22]:
import xgboost as xgb
from sklearn import cross_validation
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(a, y, test_size=0.01, random_state=33)

dtrain = xgb.DMatrix(x_train,label = y_train)
dtest = xgb.DMatrix(x_test,label =y_test)


param ={'learning_rate':0.2 , 'n_estimators':1000 , 'max_depth':2 , 'min_child_weight':3 , 'gamma':0 , 'subsample':1.0  , 'sclae_pos_weight':1 , 'eta':0.05 , 'silent':0 , 'objective':'binary:logistic'}
num_round =1000
param['eval_metric']="logloss"
plst=param.items()
evalist=[(dtest,'eval'),(dtrain,'train')]
bst = xgb.train(plst,dtrain,num_round,evalist)

print (evalist)



from sklearn import cross_validation
x_train, x_test, y_train, y_test = cross_validation.train_test_split(a, y, test_size=0.01, random_state=33)

from sklearn.svm import LinearSVC
array_trainx = x_train.values
array_trainy = y_train.values.ravel()
array_testx = x_test.values
array_testy = y_test.values.ravel()

clf = LinearSVC()
model = clf.fit(array_trainx, array_trainy)
acc =clf.score(array_testx, array_testy)

print ('SVM:',acc)


from sklearn import cross_validation
x_train, x_test, y_train, y_test = cross_validation.train_test_split(a, y, test_size=0.01, random_state=33)

from sklearn.ensemble import RandomForestClassifier

array_trainx = x_train.values
array_trainy = y_train.values.ravel()
array_testx = x_test.values
array_testy = y_test.values.ravel()

clf = RandomForestClassifier()
model = clf.fit(array_trainx, array_trainy)
acc =clf.score(array_testx, array_testy)

print ('RandomForest:',acc)

[0]	eval-logloss:0.670993	train-logloss:0.673566
[1]	eval-logloss:0.660706	train-logloss:0.659564
[2]	eval-logloss:0.651337	train-logloss:0.648516
[3]	eval-logloss:0.643025	train-logloss:0.639646
[4]	eval-logloss:0.625563	train-logloss:0.626275
[5]	eval-logloss:0.617909	train-logloss:0.617477
[6]	eval-logloss:0.615312	train-logloss:0.611405
[7]	eval-logloss:0.61505	train-logloss:0.604212
[8]	eval-logloss:0.599472	train-logloss:0.593542
[9]	eval-logloss:0.601257	train-logloss:0.588595
[10]	eval-logloss:0.599594	train-logloss:0.584594
[11]	eval-logloss:0.593899	train-logloss:0.579473
[12]	eval-logloss:0.583326	train-logloss:0.572152
[13]	eval-logloss:0.58249	train-logloss:0.568862
[14]	eval-logloss:0.581364	train-logloss:0.564521
[15]	eval-logloss:0.574629	train-logloss:0.55951
[16]	eval-logloss:0.570794	train-logloss:0.555903
[17]	eval-logloss:0.56878	train-logloss:0.553472
[18]	eval-logloss:0.562256	train-logloss:0.547564
[19]	eval-logloss:0.560345	train-logloss:0.545037
[20]	eval-logl

[164]	eval-logloss:0.462869	train-logloss:0.424898
[165]	eval-logloss:0.462996	train-logloss:0.424472
[166]	eval-logloss:0.462997	train-logloss:0.424219
[167]	eval-logloss:0.463053	train-logloss:0.424125
[168]	eval-logloss:0.462204	train-logloss:0.423993
[169]	eval-logloss:0.46197	train-logloss:0.423815
[170]	eval-logloss:0.463489	train-logloss:0.423438
[171]	eval-logloss:0.464769	train-logloss:0.423016
[172]	eval-logloss:0.464585	train-logloss:0.422866
[173]	eval-logloss:0.464587	train-logloss:0.422726
[174]	eval-logloss:0.464193	train-logloss:0.422406
[175]	eval-logloss:0.464042	train-logloss:0.422259
[176]	eval-logloss:0.463551	train-logloss:0.42211
[177]	eval-logloss:0.463423	train-logloss:0.421966
[178]	eval-logloss:0.463424	train-logloss:0.421831
[179]	eval-logloss:0.46275	train-logloss:0.421278
[180]	eval-logloss:0.46062	train-logloss:0.420825
[181]	eval-logloss:0.459834	train-logloss:0.4203
[182]	eval-logloss:0.459836	train-logloss:0.420118
[183]	eval-logloss:0.461238	train-log

[326]	eval-logloss:0.478565	train-logloss:0.378748
[327]	eval-logloss:0.478804	train-logloss:0.378554
[328]	eval-logloss:0.480017	train-logloss:0.378364
[329]	eval-logloss:0.479553	train-logloss:0.378104
[330]	eval-logloss:0.480496	train-logloss:0.377865
[331]	eval-logloss:0.481315	train-logloss:0.377677
[332]	eval-logloss:0.481555	train-logloss:0.377458
[333]	eval-logloss:0.483294	train-logloss:0.377032
[334]	eval-logloss:0.483357	train-logloss:0.376782
[335]	eval-logloss:0.485414	train-logloss:0.376526
[336]	eval-logloss:0.483218	train-logloss:0.376288
[337]	eval-logloss:0.484273	train-logloss:0.37609
[338]	eval-logloss:0.484446	train-logloss:0.375864
[339]	eval-logloss:0.484147	train-logloss:0.375679
[340]	eval-logloss:0.484135	train-logloss:0.375611
[341]	eval-logloss:0.484781	train-logloss:0.375561
[342]	eval-logloss:0.484782	train-logloss:0.375455
[343]	eval-logloss:0.484368	train-logloss:0.375212
[344]	eval-logloss:0.483418	train-logloss:0.374789
[345]	eval-logloss:0.483418	trai

[488]	eval-logloss:0.489027	train-logloss:0.345154
[489]	eval-logloss:0.489507	train-logloss:0.344929
[490]	eval-logloss:0.489204	train-logloss:0.344732
[491]	eval-logloss:0.489402	train-logloss:0.344605
[492]	eval-logloss:0.490275	train-logloss:0.344413
[493]	eval-logloss:0.490147	train-logloss:0.344367
[494]	eval-logloss:0.490148	train-logloss:0.344242
[495]	eval-logloss:0.49015	train-logloss:0.344097
[496]	eval-logloss:0.490151	train-logloss:0.343918
[497]	eval-logloss:0.491222	train-logloss:0.343693
[498]	eval-logloss:0.491453	train-logloss:0.343543
[499]	eval-logloss:0.491449	train-logloss:0.34329
[500]	eval-logloss:0.489973	train-logloss:0.343134
[501]	eval-logloss:0.488842	train-logloss:0.342927
[502]	eval-logloss:0.487649	train-logloss:0.342772
[503]	eval-logloss:0.487613	train-logloss:0.342476
[504]	eval-logloss:0.487603	train-logloss:0.342355
[505]	eval-logloss:0.487848	train-logloss:0.342184
[506]	eval-logloss:0.487783	train-logloss:0.341837
[507]	eval-logloss:0.487976	train

[650]	eval-logloss:0.495983	train-logloss:0.317131
[651]	eval-logloss:0.496015	train-logloss:0.317092
[652]	eval-logloss:0.495208	train-logloss:0.316893
[653]	eval-logloss:0.494271	train-logloss:0.316749
[654]	eval-logloss:0.493787	train-logloss:0.316596
[655]	eval-logloss:0.492649	train-logloss:0.316423
[656]	eval-logloss:0.49265	train-logloss:0.316281
[657]	eval-logloss:0.49364	train-logloss:0.316114
[658]	eval-logloss:0.493633	train-logloss:0.316027
[659]	eval-logloss:0.494727	train-logloss:0.3158
[660]	eval-logloss:0.494536	train-logloss:0.315666
[661]	eval-logloss:0.494384	train-logloss:0.315515
[662]	eval-logloss:0.494381	train-logloss:0.315454
[663]	eval-logloss:0.494233	train-logloss:0.315246
[664]	eval-logloss:0.494173	train-logloss:0.315098
[665]	eval-logloss:0.494015	train-logloss:0.314885
[666]	eval-logloss:0.495597	train-logloss:0.314716
[667]	eval-logloss:0.495049	train-logloss:0.314523
[668]	eval-logloss:0.494669	train-logloss:0.314397
[669]	eval-logloss:0.494346	train-l

[812]	eval-logloss:0.504041	train-logloss:0.292031
[813]	eval-logloss:0.504049	train-logloss:0.291899
[814]	eval-logloss:0.506168	train-logloss:0.291733
[815]	eval-logloss:0.506171	train-logloss:0.291608
[816]	eval-logloss:0.507034	train-logloss:0.2915
[817]	eval-logloss:0.507055	train-logloss:0.29138
[818]	eval-logloss:0.507057	train-logloss:0.291302
[819]	eval-logloss:0.507032	train-logloss:0.291092
[820]	eval-logloss:0.506617	train-logloss:0.290969
[821]	eval-logloss:0.506121	train-logloss:0.290807
[822]	eval-logloss:0.506231	train-logloss:0.290723
[823]	eval-logloss:0.505411	train-logloss:0.29053
[824]	eval-logloss:0.504821	train-logloss:0.290369
[825]	eval-logloss:0.504843	train-logloss:0.290244
[826]	eval-logloss:0.504957	train-logloss:0.290083
[827]	eval-logloss:0.504852	train-logloss:0.290047
[828]	eval-logloss:0.505072	train-logloss:0.289891
[829]	eval-logloss:0.50617	train-logloss:0.289696
[830]	eval-logloss:0.505788	train-logloss:0.289514
[831]	eval-logloss:0.506662	train-lo

[974]	eval-logloss:0.518529	train-logloss:0.270019
[975]	eval-logloss:0.518149	train-logloss:0.269872
[976]	eval-logloss:0.517558	train-logloss:0.269712
[977]	eval-logloss:0.518521	train-logloss:0.269568
[978]	eval-logloss:0.51966	train-logloss:0.269433
[979]	eval-logloss:0.520214	train-logloss:0.269285
[980]	eval-logloss:0.519563	train-logloss:0.269159
[981]	eval-logloss:0.520214	train-logloss:0.269039
[982]	eval-logloss:0.52015	train-logloss:0.269016
[983]	eval-logloss:0.520163	train-logloss:0.268926
[984]	eval-logloss:0.519888	train-logloss:0.268845
[985]	eval-logloss:0.519997	train-logloss:0.268744
[986]	eval-logloss:0.519982	train-logloss:0.268536
[987]	eval-logloss:0.519527	train-logloss:0.268411
[988]	eval-logloss:0.520534	train-logloss:0.268278
[989]	eval-logloss:0.520521	train-logloss:0.268144
[990]	eval-logloss:0.520595	train-logloss:0.268061
[991]	eval-logloss:0.520593	train-logloss:0.267954
[992]	eval-logloss:0.521238	train-logloss:0.267837
[993]	eval-logloss:0.520784	train

In [19]:
logreg = LogisticRegression()
logreg.fit(x_train, y_train.values.ravel())
acc_log = round(logreg.score(x_test, y_test) * 100, 2)
acc_log

61.280000000000001

In [20]:
coeff_df = pd.DataFrame(a.columns.delete(0))
coeff_df.columns = ['Feature']
coeff_df["Correlation"] = pd.Series(logreg.coef_[0])

coeff_df.sort_values(by='Correlation', ascending=True)

,Feature,Correlation
35,login_times_1M,-2.949230e-01
34,ordertype_top1,-4.870527e-02
37,login_times_3M,-5.841713e-03
14,outpkg_network_fee_1M,-2.499484e-03
13,outpkg_voice_fee_3M,-1.851798e-03
29,toll_call_fee_1M,-1.096345e-03
38,read_times_1M,-8.946440e-04
16,outpkg_network_fee_3M,-7.940230e-04
17,outpkg_mem_fee_1M,-6.953152e-04
7,reduce_fee_3M,-5.272873e-04
